In [88]:
import pandas as pd
import dateutil.parser as dateparser
import datetime


In [89]:
# put filenames in as whichever made files you ran the original ipynb for
filenames = ["zipper-a_sent.csv"]

columns = ["uid", "sender", "recipient", "sendtime", "replytime", \
                           "replyspeed", "messagecontent", "foldername"]
data = pd.DataFrame(columns=columns)
for filename in filenames:
    df = pd.read_csv("processed_data/"+filename)
    data = pd.concat([data, df], ignore_index=True)
    # data.concat(df, ignore_index=True)
# data.set_index('uid', drop=True, inplace=True)
data = data.set_index('uid')
data
# data.size

,sender,recipient,sendtime,replytime,replyspeed,messagecontent,foldername,Unnamed: 0
uid,,,,,,,,
<23245147.1075842017323.JavaMail.evans@thyme>john_bourget@bankone.com,andy.zipper@enron.com,john_bourget@bankone.com,2002-01-07 13:21:50-08:00,-1,-1,"George Bush, minister of the double negative: ...",NaN,0.0
<31662779.1075851549023.JavaMail.evans@thyme>steve.wang@enron.com,andy.zipper@enron.com,steve.wang@enron.com,2001-10-02 05:44:29-07:00,-1,-1,Month\tVol\t\nNov-01\t0.7700\t\nDec-01\t0.6675...,NaN,1.0
<14257679.1075845424721.JavaMail.evans@thyme>michael.bridges@enron.com,andy.zipper@enron.com,michael.bridges@enron.com,2001-06-06 12:53:34-07:00,-1,-1,"What is, exactly, the state of play here ? I n...",NaN,2.0
<33402920.1075845423137.JavaMail.evans@thyme>kal.shah@enron.com,andy.zipper@enron.com,kal.shah@enron.com,2001-05-30 10:15:13-07:00,-1,-1,CAN YOU SEND David Lebow in IR a copy of the a...,NaN,3.0
<15663581.1075845424335.JavaMail.evans@thyme>michael.guadarrama@enron.com,andy.zipper@enron.com,michael.guadarrama@enron.com,2001-06-05 12:41:30-07:00,-1,-1,Why wouldn't we use the deal bench auction pla...,NaN,4.0
...,...,...,...,...,...,...,...,...
<30500266.1075845424177.JavaMail.evans@thyme>daniel.diamond@enron.com,andy.zipper@enron.com,daniel.diamond@enron.com,2001-06-04 13:26:32-07:00,-1,-1,Let's table this for now. Too much going on.\n...,NaN,434.0
<33433088.1075845423768.JavaMail.evans@thyme>greg.piper@enron.com,andy.zipper@enron.com,greg.piper@enron.com,2001-06-04 07:41:33-07:00,2001-06-04 08:55:00+00:00,0 days 05:46:33,\nThis is where we are with banks for fx produ...,NaN,435.0
<12146514.1075845422622.JavaMail.evans@thyme>bob.shults@enron.com,andy.zipper@enron.com,bob.shults@enron.com,2001-05-24 14:16:41-07:00,2001-05-23 10:45:00+00:00,1 days 10:31:41,\nPlease call Joe Robertson of enron Metals in...,NaN,436.0


In [90]:
data = data[data['replytime'] != '-1']
data.head()

,sender,recipient,sendtime,replytime,replyspeed,messagecontent,foldername,Unnamed: 0
uid,,,,,,,,
<14059736.1075859061228.JavaMail.evans@thyme>aazccb@swbell.net,andy.zipper@enron.com,aazccb@swbell.net,2001-11-30 14:23:03-08:00,2001-11-30 15:46:00+00:00,0 days 06:37:03,\n\n -----Original Message-----\nFrom: \tRosta...,NaN,20.0
<1304483.1075845422644.JavaMail.evans@thyme>bob.shults@enron.com,andy.zipper@enron.com,bob.shults@enron.com,2001-05-24 14:17:36-07:00,2001-05-22 07:01:00+00:00,2 days 14:16:36,\nthis is the underlying deal. it needs work a...,NaN,29.0
<25351843.1075845424699.JavaMail.evans@thyme>greg.piper@enron.com,andy.zipper@enron.com,greg.piper@enron.com,2001-06-06 12:50:54-07:00,2001-06-06 10:07:00+00:00,0 days 09:43:54,Mike's thoughts on subject...\n\n -----Origina...,NaN,30.0
<12143138.1075859061162.JavaMail.evans@thyme>jeff.golden@enron.com,andy.zipper@enron.com,jeff.golden@enron.com,2001-11-30 13:17:27-08:00,2001-11-30 15:03:00+00:00,0 days 06:14:27,\n\n -----Original Message-----\nFrom: \tAdams...,NaN,43.0
<4255833.1075851548490.JavaMail.evans@thyme>ina.rangel@enron.com,andy.zipper@enron.com,ina.rangel@enron.com,2001-10-10 08:15:20-07:00,2001-10-05 09:56:00+00:00,5 days 05:19:20,"\nIna,\n\nWhen was this, or was it ever, suppo...",NaN,45.0


In [104]:
data.recipient.unique()
data_dict = data.to_dict('index')
data_dict

analyzed_dict = {}
for key in data_dict.keys():
    row = data_dict[key]
    
    replyspeed = pd.Timedelta(row['replyspeed']).total_seconds()
    sender = row['sender']
    recipient = row['recipient']
    # replyspeed = row['replyspeed']
    if sender in analyzed_dict:
        if recipient in analyzed_dict[sender]:
            # arr = analyzed_dict[sender][recipient]
            # arr = arr.append(replyspeed)
            analyzed_dict[sender][recipient].append(replyspeed)
            print(analyzed_dict[sender])
            print("updated")
        else:
            analyzed_dict[sender].update({recipient:[replyspeed]})
            print(analyzed_dict[sender])
            print(analyzed_dict[sender][recipient])
            print("added recipient")
    else:
        analyzed_dict.update({sender:{recipient:[replyspeed]}})
        print(analyzed_dict[sender])
        print("added sender")
        # analyzed_dict[row['sender']].append({row['recipient']:[replyspeed]})


        

{'aazccb@swbell.net': [23823.0]}
added sender
{'aazccb@swbell.net': [23823.0], 'bob.shults@enron.com': [224196.0]}
[224196.0]
added recipient
{'aazccb@swbell.net': [23823.0], 'bob.shults@enron.com': [224196.0], 'greg.piper@enron.com': [35034.0]}
[35034.0]
added recipient
{'aazccb@swbell.net': [23823.0], 'bob.shults@enron.com': [224196.0], 'greg.piper@enron.com': [35034.0], 'jeff.golden@enron.com': [22467.0]}
[22467.0]
added recipient
{'aazccb@swbell.net': [23823.0], 'bob.shults@enron.com': [224196.0], 'greg.piper@enron.com': [35034.0], 'jeff.golden@enron.com': [22467.0], 'ina.rangel@enron.com': [451160.0]}
[451160.0]
added recipient
{'aazccb@swbell.net': [23823.0], 'bob.shults@enron.com': [224196.0], 'greg.piper@enron.com': [35034.0], 'jeff.golden@enron.com': [22467.0], 'ina.rangel@enron.com': [451160.0], 'savita.puthigai@enron.com': [88665.0]}
[88665.0]
added recipient
{'aazccb@swbell.net': [23823.0], 'bob.shults@enron.com': [224196.0], 'greg.piper@enron.com': [35034.0, 20566.0], 'jef

{'andy.zipper@enron.com': {'aazccb@swbell.net': [23823.0,
   18132.0,
   107025.0,
   18028.0,
   522937.0],
  'bob.shults@enron.com': [224196.0, 19203.0, 18693.0, 124301.0],
  'greg.piper@enron.com': [35034.0, 20566.0, 21721.0, 18402.0, 20793.0],
  'jeff.golden@enron.com': [22467.0],
  'ina.rangel@enron.com': [451160.0, -215645051.0],
  'savita.puthigai@enron.com': [88665.0, 7055919.0],
  'larry.may@enron.com': [18178.0],
  'exec.jones@enron.com': [88431.0],
  'john_bourget@em.fcnbd.com': [388369.0],
  'john.lavorato@enron.com': [25353.0],
  'errol.mclaughlin@enron.com': [21653.0, 45229.0, 166269.0],
  'legal <.taylor@enron.com>': [18600.0],
  'michael.guadarrama@enron.com': [7059306.0],
  'bruce.garner@enron.com': [72093.0],
  'jay.webb@enron.com': [7055919.0, 953122.0, 102161.0, 57844.0, 18628.0],
  'lottyplace@yahoo.com': [876940.0],
  'john.griffith@enron.com': [23769.0],
  'lydia.cannon@enron.com': [18099.0],
  'nigel.patterson@db.com': [70725.0],
  'justin.rostant@enron.com': [7

In [106]:
new_df = pd.DataFrame(analyzed_dict)
new_df.to_csv("processed_data/analyzed_data.csv")